# 从全连接层到卷积
:label:`sec_why-conv`

到今天为止，
我们迄今为止讨论的模型
仍然是处理表格数据时的合适选择。
所谓表格，是指数据由
对应于样本的行和
对应于特征的列组成。
对于表格数据，我们可能会预期
所寻求的模式可能涉及
特征之间的相互作用，
但我们并不事先假设
这些特征如何相互作用。

有时，我们确实缺乏知识来指导更复杂架构的构建。
在这种情况下，多层感知机（MLP）
可能是我们能做到的最好的选择。
然而，对于高维感知数据，
这种无结构的网络可能会变得难以管理。

例如，让我们回到区分猫和狗的例子。
假设我们在数据收集方面做得非常彻底，
收集了一个一百万像素的照片标注数据集。
这意味着每个输入到网络的数据都有一百万维度。
即使将隐藏维度大幅减少到一千，
也需要一个具有 $10^6 \times 10^3 = 10^9$ 参数的全连接层。
除非我们有很多GPU、分布式优化的才能
以及非凡的耐心，
否则学习这个网络的参数
可能是不可行的。

细心的读者可能会基于一兆像素分辨率可能不必要的理由反对这一论点。
然而，尽管我们可以用十万像素勉强凑合，
但大小为1000的隐藏层严重低估了
学习图像良好表示所需的隐藏单元数量，
因此实际系统仍然需要数十亿个参数。
此外，通过拟合如此多的参数来学习分类器
可能需要收集庞大的数据集。
然而今天，人类和计算机都能够很好地
区分猫和狗，
这似乎与这些直觉相矛盾。
这是因为图像表现出丰富的结构，
可以被人类和机器学习模型利用。
卷积神经网络（CNN）是机器学习中的一种创造性方法，
用于利用自然图像中的一些已知结构。


## 不变性

想象一下，我们想要在图像中检测一个物体。
无论该物体在图像中的确切位置如何，
我们用来识别物体的方法都不应该过于关注这一点。
理想情况下，我们的系统应该利用这一知识。
猪通常不会飞，飞机通常也不会游泳。
然而，如果一只猪出现在图像的顶部，
我们仍然应该能够识别它。
我们可以从儿童游戏“寻找沃利”
（该游戏本身启发了许多现实生活中的模仿，如:numref:`img_waldo`所示）中获得一些灵感。
游戏由许多充满活动的混乱场景组成。
沃利会出现在每一个场景中，
通常潜伏在某个不太可能的地方。
读者的目标是找到他。
尽管他的服装很有特色，
但由于大量的干扰，
这可能出奇地困难。
然而，*沃利的样子*并不取决于*沃利的位置*。
我们可以用一个沃利检测器扫描图像，
该检测器可以为每个区域分配一个分数，
表示该区域包含沃利的可能性。
事实上，许多目标检测和分割算法
都是基于这种方法 :cite:`Long.Shelhamer.Darrell.2015`。
CNN系统化了这种*空间不变性*的概念，
利用它以较少的参数学习有用的表示。

![你能找到沃利吗（图片由William Murphy (Infomatique)提供）?](../img/waldo-football.jpg)
:width:`400px`
:label:`img_waldo`

我们现在可以通过列举一些设计指南
使这些直觉更加具体，
以指导适合计算机视觉的神经网络架构的设计：

1. 在最早的几层中，我们的网络
   应该对出现在图像中任何位置的相同区域
   作出类似的响应。这个原则称为*平移不变性*（或*平移等变性*）。
1. 网络的最早几层应专注于局部区域，
   而不关心远处区域的内容。这是*局部性*原则。
   最终，这些局部表示可以聚合起来
   以在整个图像级别进行预测。
1. 随着我们深入，更深的层应该能够捕获
   图像的长距离特征，
   类似于自然界中的高级视觉。

让我们看看这如何转化为数学。


## 约束MLP

首先，我们可以考虑一个MLP
其输入是二维图像 $\mathbf{X}$，
它们的直接隐藏表示
$\mathbf{H}$ 也表示为矩阵（在代码中是二维张量），
其中 $\mathbf{X}$ 和 $\mathbf{H}$ 具有相同的形状。
让我们仔细思考一下。
我们现在设想不仅输入，
而且隐藏表示也具有空间结构。

令 $[\mathbf{X}]_{i, j}$ 和 $[\mathbf{H}]_{i, j}$ 分别表示
输入图像和隐藏表示
在位置 $(i,j)$ 处的像素。
因此，为了使每个隐藏单元
都能接收来自每个输入像素的信息，
我们将从使用权重矩阵
（如我们在之前的MLP中所做的那样）
转变为将参数表示为四阶权重张量 $\mathsf{W}$。
假设 $\mathbf{U}$ 包含偏置，
我们可以正式表达全连接层为

$$\begin{aligned} \left[\mathbf{H}\right]_{i, j} &= [\mathbf{U}]_{i, j} + \sum_k \sum_l[\mathsf{W}]_{i, j, k, l}  [\mathbf{X}]_{k, l}\\ &=  [\mathbf{U}]_{i, j} +
\sum_a \sum_b [\mathsf{V}]_{i, j, a, b}  [\mathbf{X}]_{i+a, j+b}.\end{aligned}$$

从 $\mathsf{W}$ 到 $\mathsf{V}$ 的转换目前完全是形式上的，
因为两个四阶张量之间存在一一对应关系。
我们只是重新索引下标 $(k, l)$，
使得 $k = i+a$ 和 $l = j+b$。
换句话说，我们设置 $[\mathsf{V}]_{i, j, a, b} = [\mathsf{W}]_{i, j, i+a, j+b}$。
下标 $a$ 和 $b$ 遍历正负偏移，
覆盖整个图像。
对于隐藏表示 $[\mathbf{H}]_{i, j}$ 中的任何给定位置 ($i$, $j$)，
我们通过围绕 $(i, j)$ 为中心的 $x$ 中的像素求和，
并加权 $[\mathsf{V}]_{i, j, a, b}$ 来计算其值。
在继续之前，让我们考虑在这种参数化下单层所需的总参数数量：一个 $1000 \times 1000$ 的图像（1兆像素）映射到一个 $1000 \times 1000$ 的隐藏表示。
这需要 $10^{12}$ 个参数，远远超出了当前计算机的处理能力。

### 平移不变性

现在让我们引入上面建立的第一个原则：
平移不变性 :cite:`Zhang.ea.1988`。
这意味着输入 $\mathbf{X}$ 的移动
应该只导致隐藏表示 $\mathbf{H}$ 的移动。
只有当 $\mathsf{V}$ 和 $\mathbf{U}$ 实际上不依赖于 $(i, j)$ 时，这才可能实现。
因此，我们有 $[\mathsf{V}]_{i, j, a, b} = [\mathbf{V}]_{a, b}$ 且 $\mathbf{U}$ 是一个常数，比如说 $u$。
结果，我们可以简化 $\mathbf{H}$ 的定义：

$$[\mathbf{H}]_{i, j} = u + \sum_a\sum_b [\mathbf{V}]_{a, b}  [\mathbf{X}]_{i+a, j+b}.$$


这是一个*卷积*！
我们实际上是在用系数 $[\mathbf{V}]_{a, b}$
对位置 $(i+a, j+b)$ 附近的像素
进行加权，以获得 $[\mathbf{H}]_{i, j}$ 的值。
请注意，由于 $[\mathbf{V}]_{a, b}$ 不再依赖于图像中的位置，
因此它所需的系数比 $[\mathsf{V}]_{i, j, a, b}$ 少得多。
因此，所需参数的数量不再是 $10^{12}$ 而是一个更合理的 $4 \times 10^6$：
我们仍然依赖于 $a, b \in (-1000, 1000)$。
简而言之，我们取得了重大进展。
时间延迟神经网络（TDNNs）是最早利用这一思想的例子之一 :cite:`Waibel.Hanazawa.Hinton.ea.1989`。

### 局部性

现在让我们引入第二个原则：局部性。
正如上面所激励的，我们认为我们不需要
远离位置 $(i, j)$
就可以获取相关信息
来评估 $[\mathbf{H}]_{i, j}$ 发生的情况。
这意味着在范围 $|a|> \Delta$ 或 $|b| > \Delta$ 之外，
我们应该设置 $[\mathbf{V}]_{a, b} = 0$。
等价地，我们可以重写 $[\mathbf{H}]_{i, j}$ 为

$$[\mathbf{H}]_{i, j} = u + \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} [\mathbf{V}]_{a, b}  [\mathbf{X}]_{i+a, j+b}.$$
:eqlabel:`eq_conv-layer`

这将参数数量从 $4 \times 10^6$ 减少到 $4 \Delta^2$，
其中 $\Delta$ 通常小于 $10$。
因此，我们将参数数量减少了四个数量级。
注意，:eqref:`eq_conv-layer`，
简而言之就是所谓的*卷积层*。
*卷积神经网络*（CNNs）
是一类包含卷积层的特殊神经网络。
在深度学习研究社区中，
$\mathbf{V}$ 被称为*卷积核*、
*滤波器*或简单地称为层的*权重*，
这些都是可学习的参数。

虽然以前我们可能需要数十亿个参数
来表示图像处理网络中的单个层，
但现在我们通常只需要几百个参数，
而不会改变输入或隐藏表示的维度。
这种参数数量的急剧减少的代价是，
我们的特征现在是平移不变的，
并且每一层在确定每个隐藏激活的值时
只能结合局部信息。
所有的学习都依赖于施加归纳偏差。
当这种偏差与现实一致时，
我们就能得到样本高效的模型，
能够很好地泛化到未见过的数据。
但当然，如果这些偏差与现实不符，
例如，如果图像实际上不是平移不变的，
我们的模型甚至可能难以拟合训练数据。

这种参数数量的显著减少使我们达到最后一个期望，
即更深层的层应该表示更大和更复杂的图像方面。
这可以通过交错非线性和卷积层来实现。 

## 卷积

让我们简要回顾一下为什么:eqref:`eq_conv-layer`被称为卷积。
在数学中，两个函数的*卷积* :cite:`Rudin.1973`，
比如 $f, g: \mathbb{R}^d \to \mathbb{R}$ 定义为

$$(f * g)(\mathbf{x}) = \int f(\mathbf{z}) g(\mathbf{x}-\mathbf{z}) d\mathbf{z}.$$

也就是说，当我们测量 $f$ 和 $g$ 之间的重叠时，
其中一个函数被“翻转”并移动了 $\mathbf{x}$。
每当对象是离散的时候，积分就变成了求和。
例如，对于向量集合
平方可和的无限维向量
其索引运行在 $\mathbb{Z}$ 上，我们得到以下定义：

$$(f * g)(i) = \sum_a f(a) g(i-a).$$

对于二维张量，我们有一个相应的求和
$f$ 的索引为 $(a, b)$ 而 $g$ 的索引为 $(i-a, j-b)$：

$$(f * g)(i, j) = \sum_a\sum_b f(a, b) g(i-a, j-b).$$
:eqlabel:`eq_2d-conv-discrete`

这看起来类似于:eqref:`eq_conv-layer`，但有一个主要区别。
我们使用的是差值而不是 $(i+a, j+b)$。
不过，请注意，这种区别主要是形式上的，
因为我们总是可以在:eqref:`eq_conv-layer` 和:eqref:`eq_2d-conv-discrete` 之间匹配符号。
我们在:eqref:`eq_conv-layer` 中的原始定义更恰当地
描述了一个*互相关*。
我们将在下一节中回到这个问题。


## 通道
:label:`subsec_why-conv-channels`

回到我们的沃利检测器，让我们看看这看起来像什么。
卷积层选取给定大小的窗口
并根据滤波器 $\mathsf{V}$ 对强度进行加权，
如:numref:`fig_waldo_mask` 所示。
我们可能希望学习一个模型，
以便在“沃利性”最高的地方，
我们在隐藏层表示中找到一个峰值。

![检测沃利（图片由William Murphy (Infomatique)提供）。](../img/waldo-mask.jpg)
:width:`400px`
:label:`fig_waldo_mask`

这里有一个问题。
到目前为止，我们忽略了图像由三个通道组成：
红色、绿色和蓝色。
总之，图像不是二维对象，
而是第三阶张量，
具有高度、宽度和通道，
例如，形状为 $1024 \times 1024 \times 3$ 像素。
虽然前两个轴涉及空间关系，
但第三个轴可以被视为
为每个像素位置分配一个多维表示。
因此，我们将 $\mathsf{X}$ 索引为 $[\mathsf{X}]_{i, j, k}$。
卷积滤波器必须相应地调整。
与其使用 $[\mathbf{V}]_{a,b}$，我们现在有了 $[\mathsf{V}]_{a,b,c}$。

此外，正如我们的输入由第三阶张量组成一样，
将隐藏表示也表示为第三阶张量 $\mathsf{H}$ 也是个好主意。
换句话说，除了每个空间位置只有一个隐藏表示外，
我们希望每个空间位置都有一个完整的隐藏表示向量。
我们可以认为隐藏表示是由
堆叠在彼此之上的多个二维网格组成的。
与输入类似，这些有时被称为*通道*。
它们有时也被称为*特征图*，
因为每张图都提供了
为后续层学习的空间化特征集。
直观地说，你可能会想象在接近输入的较低层中，
某些通道可能专门用于识别边缘，
而其他通道可能用于识别纹理。

为了支持输入（$\mathsf{X}$）和隐藏表示（$\mathsf{H}$）中的多个通道，
我们可以为 $\mathsf{V}$ 添加第四个坐标：$[\mathsf{V}]_{a, b, c, d}$。
将所有内容放在一起，我们有：

$$[\mathsf{H}]_{i,j,d} = \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} \sum_c [\mathsf{V}]_{a, b, c, d} [\mathsf{X}]_{i+a, j+b, c},$$
:eqlabel:`eq_conv-layer-channels`

其中 $d$ 索引隐藏表示 $\mathsf{H}$ 中的输出通道。
随后的卷积层将继续
将第三阶张量 $\mathsf{H}$ 作为输入。
我们采用:eqref:`eq_conv-layer-channels`，
因为它具有普遍性，
作为多通道卷积层的定义，
其中 $\mathsf{V}$ 是层的核或滤波器。

我们还需要解决许多操作。
例如，我们需要弄清楚如何
将所有隐藏表示组合成单一输出，
例如，图像中是否有沃利。
我们还需要决定如何高效地计算，
如何组合多个层，
适当的激活函数，
以及如何做出合理的设计选择
以产生在实践中有效的网络。
我们在本章的其余部分将讨论这些问题。

## 总结和讨论

在本节中，我们从第一原理推导了卷积神经网络的结构。
虽然不清楚这是否是发明CNN的途径，
但在应用合理的原理来处理
图像处理和计算机视觉算法应该如何工作时，
特别是在低层次上，
知道它们是*正确*的选择是令人满意的。
特别是，图像中的平移不变性意味着
图像的所有区域都将受到同样的对待。
局部性意味着只使用一个小邻域的像素
来计算相应的隐藏表示。
最早的关于CNN的参考文献之一是Neocognitron :cite:`Fukushima.1982`。

我们在推理中遇到的第二个原则是
如何在不对模型的表达能力造成限制的情况下
减少函数类中的参数数量，
至少在模型满足某些假设的情况下。
我们看到由于这种限制导致了复杂性的显著降低，
将计算和统计上不可行的问题转化为可处理的模型。

增加通道使我们能够恢复一些
由于局部性和平移不变性对卷积核的限制
而失去的复杂性。
注意到除了红色、绿色和蓝色之外
添加其他通道是很自然的。
许多卫星图像，特别是农业和气象方面的，
有几十到几百个通道，
生成高光谱图像而不是普通的RGB图像。
它们报告了多种不同波长的数据。
在接下来的部分中，我们将看到
如何有效地使用卷积来操纵它们操作的图像的维度，
如何从基于位置的表示转移到基于通道的表示，
以及如何高效地处理大量类别。

## 练习

1. 假设卷积核的大小为 $\Delta = 0$。
   证明在这种情况下，
   卷积核独立地为每个通道集实现了MLP。
   这导致了Network in Network架构 :cite:`Lin.Chen.Yan.2013`。
1. 音频数据通常表示为一维序列。
    1. 何时可能希望对音频施加局部性和平移不变性？
    1. 推导音频的卷积操作。
    1. 你能用与计算机视觉相同的工具处理音频吗？提示：使用频谱图。
1. 为什么平移不变性最终可能不是一个好主意？举个例子。
1. 你认为卷积层也可能适用于文本数据吗？
   语言中可能会遇到哪些问题？
1. 当物体位于图像边界时，卷积会发生什么？
1. 证明卷积是对称的，即 $f * g = g * f$。

[讨论](https://discuss.d2l.ai/t/64)